Playing musics from models

In [1]:
from keras.models import Model, load_model
import numpy as np
import utils as ut
import PreProcessor as pp
from mido import Message, MetaMessage, MidiFile, MidiTrack, bpm2tempo, second2tick

In [2]:
seq_length = 30

#dataset = pp.load_dataset("../adl-piano-midi")
dataset = pp.load_dataset("C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity = pp.dicts_from_songs(ClassicSongs,velocities=True)
time_range = pp.ranges_from_songs(ClassicSongs)

n_Channels, n_Notes, n_Velocities = len(channel_to_ind), len(note_to_ind), len(velocity_to_ind)

print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])


Number of channels   : 12 
Number of notes      : 105 
Number of velocities : 128 
Time range           : 0.0 190.285


In [3]:
LSTM_model = ut.create_model("LSTM",250,n_Channels,n_Notes,n_Velocities,time_range)
LSTM_model.load_weights("LSTM_model.h5")

In [4]:
seq_length = 30
n_times = 2

In [5]:
X_Channels, X_Notes, X_Velocities, X_Times, y_Channels, y_Notes, y_Velocities, y_Times, Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times, Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times, Test_X_Channels, Test_X_Notes, Test_X_Velocities, Test_X_Times, Test_y_Channels, Test_y_Notes, Test_y_Velocities, Test_y_Times = ut.prep_data(seq_length, ClassicSongs,n_Channels,n_Notes,n_Velocities,channel_to_ind,note_to_ind,velocity_to_ind)

In [6]:
gen = ut.generate(LSTM_model,[X_Channels[2:3,:,:], X_Notes[2:3,:,:], X_Velocities[2:3,:,:], X_Times[2:3,:,:]],30,ind_to_channel,ind_to_note,mode="SoftMax",T=0.8)
gen

1/1 [==============================] - 0s 31ms/step


[[0, 45, 0, 0.11623852],
 [0, 64, 0, 2.9192522],
 [0, 67, 0, 2.2710068],
 [0, 70, 1, 1.9492811],
 [0, 76, 1, 1.9983574],
 [0, 75, 1, 1.5607165],
 [0, 76, 1, 0.8395762],
 [0, 50, 1, 1.44875],
 [0, 64, 1, 0.7806779],
 [0, 70, 1, 1.1210564],
 [0, 68, 1, 1.9203656],
 [0, 70, 1, 1.1909192],
 [0, 67, 1, 1.919628],
 [0, 70, 1, 1.0955892],
 [0, 72, 1, 1.9212233],
 [0, 75, 1, 1.1700073],
 [0, 76, 1, 0.86381245],
 [0, 65, 1, 1.4460328],
 [0, 63, 1, 0.81066954],
 [0, 66, 1, 0.84031224],
 [0, 68, 1, 1.2515783],
 [0, 56, 1, 1.2740638],
 [0, 60, 1, 0.79321545],
 [0, 60, 1, 1.1212988],
 [0, 63, 1, 1.0473826],
 [0, 68, 1, 0.7815831],
 [0, 70, 1, 1.4437771],
 [0, 73, 1, 1.9327856],
 [0, 76, 1, 1.4802798],
 [0, 77, 1, 1.4673667]]

In [7]:
channels,notes,velocities,times = ut.create_midi(gen)
print(channels)
time_ = [int(l*960) for l in times]
print(notes)
print(velocities)
print(times)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[45, 64, 67, 70, 76, 75, 76, 50, 64, 70, 68, 70, 67, 70, 72, 75, 76, 65, 63, 66, 68, 56, 60, 60, 63, 68, 70, 73, 76, 77]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.11623852, 2.9192522, 2.2710068, 1.9492811, 1.9983574, 1.5607165, 0.8395762, 1.44875, 0.7806779, 1.1210564, 1.9203656, 1.1909192, 1.919628, 1.0955892, 1.9212233, 1.1700073, 0.86381245, 1.4460328, 0.81066954, 0.84031224, 1.2515783, 1.2740638, 0.79321545, 1.1212988, 1.0473826, 0.7815831, 1.4437771, 1.9327856, 1.4802798, 1.4673667]


In [8]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(MetaMessage('key_signature', key='Dm'))
#track.append(MetaMessage('set_tempo', tempo=bpm2tempo(960)))
track.append(MetaMessage('time_signature', numerator=6, denominator=8))

for i in range(len(channels)):
    track.append(Message('note_on', channel=channels[i], note=notes[i], velocity=velocities[i], time=time_[i]))
track.append(MetaMessage('end_of_track'))
mid.save('new_song.mid')

In [10]:
import pygame
pygame.init()
pygame.mixer.music.load("new_song.mid")
pygame.mixer.music.play()